# Advent of Code

## 2017-012-018
## 2017 018

https://adventofcode.com/2017/day/18

In [2]:
# Parse the input file into a list of instructions
def parse_instructions(file_path):
    with open(file_path, 'r') as file:
        return [line.strip().split() for line in file.readlines()]

# Load instructions
instructions = parse_instructions('input.txt')

# Initialize registers and state
registers = {}
last_sound = None
instruction_pointer = 0

def get_value(x):
    """Returns the value of a register or a direct integer."""
    if x.isalpha():  # It's a register
        return registers.get(x, 0)
    return int(x)  # It's an integer

# Execute instructions until rcv is triggered with a non-zero value
while 0 <= instruction_pointer < len(instructions):
    parts = instructions[instruction_pointer]
    cmd = parts[0]
    x = parts[1]
    y = parts[2] if len(parts) > 2 else None
    
    if cmd == "snd":
        # Play sound
        last_sound = get_value(x)
    elif cmd == "set":
        # Set register
        registers[x] = get_value(y)
    elif cmd == "add":
        # Add to register
        registers[x] = get_value(x) + get_value(y)
    elif cmd == "mul":
        # Multiply register
        registers[x] = get_value(x) * get_value(y)
    elif cmd == "mod":
        # Modulo operation
        registers[x] = get_value(x) % get_value(y)
    elif cmd == "rcv":
        # Recover frequency if non-zero
        if get_value(x) != 0:
            break
    elif cmd == "jgz":
        # Jump if greater than zero
        if get_value(x) > 0:
            instruction_pointer += get_value(y)
            continue  # Skip the normal increment
    
    # Increment instruction pointer
    instruction_pointer += 1

last_sound


1187

In [3]:
from collections import defaultdict, deque

# Load instructions for both programs
instructions = parse_instructions('input.txt')

# Initialize program states
programs = [
    {"registers": defaultdict(int), "queue": deque(), "pointer": 0, "send_count": 0, "waiting": False},
    {"registers": defaultdict(int), "queue": deque(), "pointer": 0, "send_count": 0, "waiting": False}
]
programs[0]["registers"]["p"] = 0
programs[1]["registers"]["p"] = 1

def get_value(state, x):
    """Returns the value of a register or a direct integer."""
    if x.isalpha():  # It's a register
        return state["registers"][x]
    return int(x)  # It's an integer

# Function to execute a single instruction for a program
def execute_instruction(program_id, other_program_id):
    state = programs[program_id]
    other_state = programs[other_program_id]
    instructions_count = len(instructions)
    
    if not (0 <= state["pointer"] < instructions_count):
        return False  # Terminated
    
    parts = instructions[state["pointer"]]
    cmd = parts[0]
    x = parts[1]
    y = parts[2] if len(parts) > 2 else None
    
    if cmd == "snd":
        # Send value to the other program's queue
        value = get_value(state, x)
        other_state["queue"].append(value)
        state["send_count"] += 1
    elif cmd == "set":
        # Set register
        state["registers"][x] = get_value(state, y)
    elif cmd == "add":
        # Add to register
        state["registers"][x] += get_value(state, y)
    elif cmd == "mul":
        # Multiply register
        state["registers"][x] *= get_value(state, y)
    elif cmd == "mod":
        # Modulo operation
        state["registers"][x] %= get_value(state, y)
    elif cmd == "rcv":
        # Receive from queue
        if state["queue"]:
            state["registers"][x] = state["queue"].popleft()
            state["waiting"] = False
        else:
            state["waiting"] = True
            return False  # Blocked
    elif cmd == "jgz":
        # Jump if greater than zero
        if get_value(state, x) > 0:
            state["pointer"] += get_value(state, y)
            return True  # Skip the normal increment
    
    state["pointer"] += 1
    return True

# Simulate the two programs
while True:
    p0_active = execute_instruction(0, 1)
    p1_active = execute_instruction(1, 0)
    
    # Check if both programs are waiting or terminated
    if not p0_active and not p1_active:
        break

# Get the send count for program 1
programs[1]["send_count"]


5969